In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
%matplotlib inline

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int,
              'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float,
              'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float,
              'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

In [3]:
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(lambda x: np.sqrt(x))
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(lambda x: np.sqrt(x))
sales['bedrooms_square'] = sales['bedrooms'].apply(lambda x: np.square(x))
sales['floors_square'] = sales['floors'].apply(lambda x: np.square(x))

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

In [6]:
model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [48]:
pd.Series(model_all.coef_, index = all_features)

bedrooms                0.000000
bedrooms_square         0.000000
bathrooms               0.000000
sqft_living           134.439314
sqft_living_sqrt        0.000000
sqft_lot                0.000000
sqft_lot_sqrt           0.000000
floors                  0.000000
floors_square           0.000000
waterfront              0.000000
view                24750.004586
condition               0.000000
grade               61749.103091
sqft_above              0.000000
sqft_basement           0.000000
yr_built               -0.000000
yr_renovated            0.000000
dtype: float64

In [8]:
model_all.intercept_

-218136.21403514093

In [9]:
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)

In [10]:
training['sqft_living_sqrt'] = training['sqft_living'].apply(lambda x: np.sqrt(x))
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(lambda x: np.sqrt(x))
training['bedrooms_square'] = training['bedrooms'].apply(lambda x: np.square(x))
training['floors_square'] = training['floors'].apply(lambda x: np.square(x))

testing['sqft_living_sqrt'] = testing['sqft_living'].apply(lambda x: np.sqrt(x))
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(lambda x: np.sqrt(x))
testing['bedrooms_square'] = testing['bedrooms'].apply(lambda x: np.square(x))
testing['floors_square'] = testing['floors'].apply(lambda x: np.square(x))

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(lambda x: np.sqrt(x))
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(lambda x: np.sqrt(x))
validation['bedrooms_square'] = validation['bedrooms'].apply(lambda x: np.square(x))
validation['floors_square'] = validation['floors'].apply(lambda x: np.square(x))

In [29]:
l1_penalty = np.logspace(1, 7, num=13)

In [30]:
class lasso:
    def __init__(self, train_x, train_y, val_x, val_y, test_x, test_y, penalty):
        self.train_x = train_x
        self.train_y = train_y
        self.test_x = test_x
        self.test_y = test_y
        self.val_x = val_x
        self.val_y = val_y
        self.penalty = penalty
        
    def fit (self):
        model = linear_model.Lasso(alpha=self.penalty, normalize=True)
        model.fit(self.train_x, self.train_y) # learn weights
        self.coff = model.coef_
        self.intercept = model.intercept_
        val_y_hat = model.predict(self.val_x)
        val_rss = self.RSS_calc(val_y_hat, self.val_y)
        test_y_hat =  model.predict(self.test_x)
        test_rss = self.RSS_calc(test_y_hat, self.test_y)
        non_zero = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
        return val_rss, test_rss, non_zero
    def RSS_calc(self, prediction, target):
        return np.sum((prediction - target)**2)
        

In [31]:
summary = pd.DataFrame(data=None, columns=['L1 Penalty', 'Validation RSS', 'Test RSS', 'Non Zero'])
index = 0
for l1 in l1_penalty:
    train_x= training[all_features]
    train_y = training['price']
    test_x= testing[all_features]
    test_y = testing['price']
    val_x= validation[all_features]
    val_y = validation['price']
    model = lasso(train_x= train_x, train_y=train_y, val_x=val_x, val_y=val_y,
                  test_x=test_x, test_y=test_y, penalty=l1)
    val_rss, test_rss, nonzero = model.fit()
    summary.loc[index] = [l1, val_rss, test_rss, nonzero]
    index+=1

summary.sort_values('Validation RSS', inplace=True)






In [32]:
summary

,L1 Penalty,Validation RSS,Test RSS,Non Zero
0,1.000000e+01,3.982133e+14,9.846740e+13,15.0
1,3.162278e+01,3.990419e+14,9.977633e+13,15.0
2,1.000000e+02,4.297916e+14,1.070207e+14,11.0
3,3.162278e+02,4.637398e+14,1.142137e+14,6.0
4,1.000000e+03,6.458987e+14,1.516941e+14,4.0
5,3.162278e+03,1.222507e+15,2.847189e+14,1.0
6,1.000000e+04,1.222507e+15,2.847189e+14,1.0
7,3.162278e+04,1.222507e+15,2.847189e+14,1.0
8,1.000000e+05,1.222507e+15,2.847189e+14,1.0
9,3.162278e+05,1.222507e+15,2.847189e+14,1.0


In [33]:
max_nonzeros = 7
l1_penalty = np.logspace(1, 4, num=20)


In [34]:
l1_penalty

array([    10.        ,     14.38449888,     20.69138081,     29.76351442,
           42.81332399,     61.58482111,     88.58667904,    127.42749857,
          183.29807108,    263.66508987,    379.26901907,    545.55947812,
          784.75997035,   1128.83789168,   1623.77673919,   2335.72146909,
         3359.81828628,   4832.93023857,   6951.92796178,  10000.        ])

In [35]:
search = pd.DataFrame(columns=['L1 Penalty', 'Validation RSS', 'Test RSS', 'Non Zero'])
index = 0
train_x= training[all_features]
train_y = training['price']
test_x= testing[all_features]
test_y = testing['price']
val_x= validation[all_features]
val_y = validation['price']
for l1 in l1_penalty:
    model = lasso(train_x= train_x, train_y=train_y, val_x=val_x, val_y=val_y,
                  test_x=test_x, test_y=test_y, penalty=l1)
    val_rss, test_rss, nonzero = model.fit()
    search.loc[index] = [l1, val_rss, test_rss, nonzero]
    index+=1

In [36]:
search

,L1 Penalty,Validation RSS,Test RSS,Non Zero
0,10.000000,3.982133e+14,9.846740e+13,15.0
1,14.384499,3.968318e+14,9.846649e+13,15.0
2,20.691381,3.962109e+14,9.869879e+13,15.0
3,29.763514,3.982155e+14,9.953588e+13,15.0
4,42.813324,4.068773e+14,1.017118e+14,13.0
5,61.584821,4.246475e+14,1.056879e+14,12.0
6,88.586679,4.279063e+14,1.065583e+14,11.0
7,127.427499,4.353747e+14,1.083210e+14,10.0
8,183.298071,4.431072e+14,1.100023e+14,7.0
9,263.665090,4.541767e+14,1.122713e+14,6.0


In [37]:
l1_min = 127.427499
l1_max = 263.665090

l1_penalty = np.linspace(l1_min,l1_max,20)

In [38]:
l1_penalty

array([ 127.427499  ,  134.59789853,  141.76829805,  148.93869758,
        156.10909711,  163.27949663,  170.44989616,  177.62029568,
        184.79069521,  191.96109474,  199.13149426,  206.30189379,
        213.47229332,  220.64269284,  227.81309237,  234.98349189,
        242.15389142,  249.32429095,  256.49469047,  263.66509   ])

In [39]:
optimal = pd.DataFrame(columns=['L1 Penalty', 'Validation RSS', 'Test RSS', 'Non Zero'])
index = 0
train_x= training[all_features]
train_y = training['price']
test_x= testing[all_features]
test_y = testing['price']
val_x= validation[all_features]
val_y = validation['price']
for l1 in l1_penalty:
    model = lasso(train_x= train_x, train_y=train_y, val_x=val_x, val_y=val_y,
                  test_x=test_x, test_y=test_y, penalty=l1)
    val_rss, test_rss, nonzero = model.fit()
    optimal.loc[index] = [l1, val_rss, test_rss, nonzero]
    index+=1

In [40]:
optimal

,L1 Penalty,Validation RSS,Test RSS,Non Zero
0,127.427499,4.353747e+14,1.083210e+14,10.0
1,134.597899,4.370092e+14,1.086736e+14,10.0
2,141.768298,4.382361e+14,1.089401e+14,8.0
3,148.938698,4.391589e+14,1.091507e+14,8.0
4,156.109097,4.400374e+14,1.093468e+14,7.0
5,163.279497,4.407775e+14,1.095060e+14,7.0
6,170.449896,4.415667e+14,1.096748e+14,7.0
7,177.620296,4.424064e+14,1.098536e+14,7.0
8,184.790695,4.432967e+14,1.100424e+14,7.0
9,191.961095,4.442398e+14,1.102417e+14,7.0


In [41]:
optimal.sort_values(['Non Zero', 'Validation RSS'], inplace=True)

In [42]:
optimal

,L1 Penalty,Validation RSS,Test RSS,Non Zero
11,206.301894,4.462689e+14,1.106679e+14,6.0
12,213.472293,4.471129e+14,1.108389e+14,6.0
13,220.642693,4.479982e+14,1.110183e+14,6.0
14,227.813092,4.489247e+14,1.112061e+14,6.0
15,234.983492,4.498925e+14,1.114023e+14,6.0
16,242.153891,4.509015e+14,1.116070e+14,6.0
17,249.324291,4.519524e+14,1.118199e+14,6.0
18,256.494690,4.530439e+14,1.120414e+14,6.0
19,263.665090,4.541767e+14,1.122713e+14,6.0
4,156.109097,4.400374e+14,1.093468e+14,7.0


In [43]:
optimal_l1 = 156.109097

In [44]:
optimal_model = linear_model.Lasso(alpha=optimal_l1, normalize=True) # set parameters
optimal_model.fit(training[all_features], training['price']) # learn weights

Lasso(alpha=156.109097, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [45]:
all_features

['bedrooms',
 'bedrooms_square',
 'bathrooms',
 'sqft_living',
 'sqft_living_sqrt',
 'sqft_lot',
 'sqft_lot_sqrt',
 'floors',
 'floors_square',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

In [47]:
pd.Series(optimal_model.coef_, index = all_features)

bedrooms                -0.000000
bedrooms_square         -0.000000
bathrooms            10610.890230
sqft_living            163.380252
sqft_living_sqrt         0.000000
sqft_lot                -0.000000
sqft_lot_sqrt           -0.000000
floors                   0.000000
floors_square            0.000000
waterfront          506451.686907
view                 41960.043550
condition                0.000000
grade               116253.553672
sqft_above               0.000000
sqft_basement            0.000000
yr_built             -2612.234878
yr_renovated             0.000000
dtype: float64